In [ ]:
from pyspark.sql.functions import col
from capstone_pipeline.main import transform_clean_timestamp, transform_clean_currency, \
    fiter_corrupt_records, transform_clean_digits, audit_log


table_name = "capstone.silver.transactions"
log_path = "/Volumes/capstone/bronze/history"

In [ ]:
dftransactions = spark.table("capstone.bronze.transactions")
display(dftransactions.columns)

# "order_id","item_id","quantity","price","order_timestamp","corrupted_flag","_ingest_timestamp","_source_file_name"

In [ ]:
dftransactions_cleaned = (dftransactions
            .drop("_rescue")
            .transform(fiter_corrupt_records, "corrupted_flag")
            .transform(transform_clean_timestamp, "order_timestamp")
            .transform(transform_clean_currency, "price")
            .transform(transform_clean_digits, "quantity")
            .withColumn(
                    "price",
                    col("price").cast("decimal(10,2)")
            )
            .withColumn(
                    "quantity",
                    col("quantity").cast("int")
            )
            .select("order_id","item_id","quantity","price","order_timestamp","_ingest_timestamp","_source_file_name"))


In [ ]:
(dftransactions_cleaned
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(table_name))


In [ ]:
audit_log(spark, table_name, log_path)